# Trajectory Curves

## Load Image

In [1]:
import cv2
import numpy as np
import math
from math import ceil
import rosbag
import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

In [2]:
# Get segmented images from the rosbag and convert to opencv Images
bag = rosbag.Bag('subset.bag')
bridge = CvBridge()
segmented_images_cv2 = []
for topic, msg, t in bag.read_messages(topics=['/segmented_image']):
    segmented_images_cv2.append(bridge.imgmsg_to_cv2(msg, "bgr8"))

In [3]:
## Get image shape , rows, columns and channels 
shape = segmented_images_cv2[0].shape
height = shape[0]
width = shape[1]-1
print(shape)

(480, 640, 3)


In [4]:
horizon = int(height * 0.4)


In [5]:
trajectory_pixels = [10711,12315,13894,9520,13894,12315,10711]

In [6]:
def is_red_pixel(image, x, y):
    if image[y, x, 2] == 255 and image[y, x, 0] == 0 and image[y, x, 1] == 0:
        return 1
    return 0

In [7]:
def center_trajectories(image, r, visualize=True):
    red_pixel_count = 0
    for y in range(height-50, horizon , -1):
        xL = int((ceil(width/2.) -r))
        xR = int((ceil(width/2.) +r))
        for x in range(xL,xR):
            red_pixel_count += is_red_pixel(image, x, y)
            if visualize and int(image[y, x, 2] == 255 and image[y, x, 0] == 0 and image[y, x, 1] == 0):
                image[y,x] = (255,255,255)
            elif int(image[y, x, 2] == 0 and image[y, x, 0] == 0 and image[y, x, 1] == 255):
                return red_pixel_count
    return red_pixel_count

In [8]:
## Right Trajectories
def right_trajectories(image, R, r, LTolerance, visualize=True):
    red_pixel_count = 0
    for y in range(height - 50, horizon , -1):
        xL = width
        xR = width
        if (R+r) * (R+r)-(y-height)*(y-height) >= 0 :
            xL = int((ceil(width/2.)+(R-r))-math.sqrt((R+r)*(R+r)-(y-height)*(y-height)))
        if (R-r) * (R-r)-(y-height)*(y-height) >= 0:
            xR = int((ceil(width/2.)+(R+r))-math.sqrt((R-r)*(R-r)-(y-height)*(y-height)))
        xL = max( min(xL,width), 0 )
        xR = max( min(xR,width), 0 )
        x_count = 0
        for x in range(xL, xR):
            red_pixel_count += is_red_pixel(image, x, y)
            if visualize and int(image[y, x, 2] == 255 and image[y, x, 0] == 0 and image[y, x, 1] == 0):
                image[y,x] = (255,255,255)
            elif int(image[y, x, 2] == 0 and image[y, x, 0] == 0 and image[y, x, 1] == 255):
                 if (x_count < LTolerance):
                    return red_pixel_count
                 else:
                    break
            x_count += 1
    return red_pixel_count

In [9]:
## Left Trajectories
def left_trajectories(image, R, r, LTolerance, visualize=True):
    red_pixel_count = 0
    for y in range(height - 50, horizon , -1):
        xL = 0
        xR = 0
        if (R-r)*(R-r)-(y-height)*(y-height) >= 0 :
            xL = int((ceil(width/2.)-(R+r))+math.sqrt((R-r)*(R-r)-(y-height)*(y-height)))
        if (R+r)*(R+r)-(y-height)*(y-height) >= 0:
            xR = int((ceil(width/2.)-(R-r))+math.sqrt((R+r)*(R+r)-(y-height)*(y-height)))
        xL = max(min(xL,width),0)
        xR = max(min(xR,width),0)
        x_count = 0
        for x in range(xR - 1, xL-1, -1):
            red_pixel_count += is_red_pixel(image, x, y)
            if visualize and int(image[y, x, 2] == 255 and image[y, x, 0] == 0 and image[y, x, 1] == 0):
                image[y,x] = (255,255,255)
            elif int(image[y, x, 2] == 0 and image[y, x, 0] == 0 and image[y, x, 1] == 255):
                 if (x_count < LTolerance):
                    return red_pixel_count
                 else:
                    break
            x_count += 1
    return red_pixel_count

In [10]:
image = segmented_images_cv2[0].copy()
image = image[:,:-1,:]
results = []
results.append(left_trajectories(image,100, 10, 10,False))
results.append(left_trajectories(image,150, 10, 10,False))
results.append(left_trajectories(image,200, 10, 10,False))
results.append(center_trajectories(image,20,False))
results.append(right_trajectories(image,200, 10, 10,False))
results.append(right_trajectories(image,150, 10, 10,False))
results.append(right_trajectories(image,100, 10, 10,False))
print results
results = np.array(results,dtype=float)/np.array(trajectory_pixels,dtype=float)
print results

[2751, 3218, 3531, 5045, 4429, 3838, 3805]
[0.25683876 0.26130735 0.25413848 0.52993697 0.31877069 0.31165246
 0.35524227]


In [11]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

s = softmax(results)
np.sum(s)

1.0

In [12]:
STEERING_RATIOS = [-1.0, -0.6, -0.3, 0, 0.3, 0.6, 1.0]


In [13]:
np.dot(results, STEERING_RATIOS)

0.14800023956870514

In [14]:
now_nsecs = rospy.get_rostime().nsecs
rospy.loginfo("Current time %i", now_nsecs)

ROSInitException: time is not initialized. Have you called init_node()?

In [15]:

image = segmented_images_cv2[0].copy()
image = image[:,:-1,:]
left_trajectories(image,100, 10, 10)
left_trajectories(image,150, 10, 10)
left_trajectories(image,200, 10, 10)
center_trajectories(image,20)
right_trajectories(image,200, 10, 10)
right_trajectories(image,150, 10, 10)
right_trajectories(image,100, 10, 10)

2149

In [16]:
for idx, image in enumerate(segmented_images_cv2):
    image_copy = image.copy()
    left_trajectories(image_copy,100, 10, 10)
    left_trajectories(image_copy,150, 10, 10)
    left_trajectories(image_copy,200, 10, 10)
    center_trajectories(image_copy,20)
    right_trajectories(image_copy,200, 10, 10)
    right_trajectories(image_copy,150, 10, 10)
    right_trajectories(image_copy,100, 10, 10)
    cv2.imwrite('traject_thunderhill/{}_traject.jpg'.format(idx), image_copy)

In [ ]:
cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()